<h1>DeviceEngine Class

Dedicated engine for device data, inherited from Core Engine. Each DeviceEngine class object will represent a unique device with its own set of processing parameters and results.

In [ ]:
from src.StreamPort.device.DeviceEngine import DeviceEngine
from src.StreamPort.core.ProcessingSettings import ProcessingSettings
from src.StreamPort.core.ProjectHeaders import ProjectHeaders

In [ ]:
settings = ProcessingSettings()

In [ ]:
base_dir = r'C:\Users\sande\Desktop\ExtractedSignals'

In [ ]:

#Creates an empty DeviceEngine object and prints it
dev = DeviceEngine(source = base_dir)
dev.print()

In [ ]:
dev.add_headers(headers = {'name': 'Pressure Curve Analysis', 'author': 'Sandeep H.'})
dev.print()

In [ ]:
dev.add_settings(settings)
dev.print()

In [ ]:
analyses = dev.read_device_spectra()

<h1>DeviceAnalysis Class

Each DeviceAnalysis object is a child of the Analysis Class. It holds the analysis details for each individual device.

In [ ]:
from src.StreamPort.device.DeviceAnalysis import DeviceAnalysis

#Creates an empty DeviceAnalysis object and prints it
devAnalysis = DeviceAnalysis()
devAnalysis.print()